In [1]:
from scipy.stats import norm
from tqdm import tqdm
import pandas as pd
import numpy as np

import sys, os

In [2]:
options = pd.read_csv("data/options.csv")
rates = pd.read_csv("data/cubic_ratemap.csv")

In [3]:
options

,date_current,ticker,expiration_date,days_to_expiry,option_id,option_type,strike_price,bid_price,option_price,ask_price,implied_volatility,volume,open_interest,stock_price,dividend_yield,tdays_to_expiry
0,2020-06-01,SPY,2020-06-01,0,SPY 2020-06-01 C145,C,145.0,160.36,139.98,161.11,596.88,3,3,305.55,1.99,0
1,2020-06-01,SPY,2020-06-01,0,SPY 2020-06-01 C150,C,150.0,155.36,139.87,156.11,571.88,1,0,305.55,1.99,0
2,2020-06-01,SPY,2020-06-01,0,SPY 2020-06-01 C160,C,160.0,145.36,144.85,146.11,524.22,2,2,305.55,1.99,0
3,2020-06-01,SPY,2020-06-01,0,SPY 2020-06-01 C165,C,165.0,140.36,139.87,141.11,500.78,2,2,305.55,1.99,0
4,2020-06-01,SPY,2020-06-01,0,SPY 2020-06-01 C180,C,180.0,125.36,119.58,126.11,435.94,10,0,305.55,1.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1064252,2020-12-15,SPY,2023-01-20,766,SPY 2023-01-20 P510,P,510.0,151.72,151.36,155.50,24.54,1,0,369.59,1.57,528
1064253,2020-12-15,SPY,2023-01-20,766,SPY 2023-01-20 P515,P,515.0,155.50,155.95,160.50,24.96,1,0,369.59,1.57,528
1064254,2020-12-15,SPY,2023-01-20,766,SPY 2023-01-20 P520,P,520.0,161.15,160.39,165.00,25.06,25,0,369.59,1.57,528
1064255,2020-12-15,SPY,2023-01-20,766,SPY 2023-01-20 P525,P,525.0,163.04,182.38,167.50,23.85,31,1,369.59,1.57,528


### Filter Options

In [4]:
moneyness = abs(options.stock_price / options.strike_price - 1)
options = options[moneyness <= 0.35]
options = options[options.bid_price != 0]
options = options[options.ask_price != 0]
options = options.merge(rates, on=['date_current', 'days_to_expiry'], how="inner")
options = options[options.tdays_to_expiry > 0]
options = options.reset_index(drop=True)

In [5]:
options.head()

,date_current,ticker,expiration_date,days_to_expiry,option_id,option_type,strike_price,bid_price,option_price,ask_price,implied_volatility,volume,open_interest,stock_price,dividend_yield,tdays_to_expiry,rate
0,2020-06-01,SPY,2020-06-03,2,SPY 2020-06-03 C230,C,230.0,75.52,75.12,75.87,139.65,11,2,305.55,1.99,2,0.002167
1,2020-06-01,SPY,2020-06-03,2,SPY 2020-06-03 C240,C,240.0,65.52,55.70,65.87,121.09,5,5,305.55,1.99,2,0.002167
2,2020-06-01,SPY,2020-06-03,2,SPY 2020-06-03 C245,C,245.0,60.52,48.46,60.87,111.91,34,34,305.55,1.99,2,0.002167
3,2020-06-01,SPY,2020-06-03,2,SPY 2020-06-03 C250,C,250.0,55.52,55.82,55.87,102.93,23,24,305.55,1.99,2,0.002167
4,2020-06-01,SPY,2020-06-03,2,SPY 2020-06-03 C256,C,256.0,49.52,48.43,49.87,92.19,26,26,305.55,1.99,2,0.002167


### Model Functions

In [17]:
def bs_price(v, S, K, T, r, q, t, M):
    
    d1 = np.log(S / K) + (r + 0.5 * v * v) * T
    d1 /= np.sqrt(T) * v
    d2 = d1 - np.sqrt(T) * v
    
    if t == 1:
        return S * np.exp(-q * T) * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    else:
        return K * np.exp(-r * T) * norm.cdf(-d2) - S * np.exp(-q * T) * norm.cdf(-d1)
    
def vega(v, S, K, T, r, q, t, M):
    
    d1 = np.log(S / K) + (r + 0.5 * v * v) * T
    d1 /= np.sqrt(T) * v
    return S * np.exp(-q * T) * norm.pdf(d1) * np.sqrt(T)

### Root Parameters

In [18]:
Ss = options.stock_price.values
Ks = options.strike_price.values
rs = options.rate.values / 100
Ts = options.tdays_to_expiry.values / 252
qs = options.dividend_yield.values / 100
ts = options.option_type.map({"C" : 1, "P" : -1}).values
Ms = (options.ask_price + options.bid_price).values * 0.5

### Root Finders

In [19]:
from scipy.optimize import newton, brent, brentq

In [24]:
def root(v, *args):
    print(bs_price(v, *args), M, v)
    return bs_price(v, *args) - M
    
ivs = []
c = 0
for S, K, T, r, q, t, M in zip(Ss, Ks, Ts, rs, qs, ts, Ms):
#     try:
#         iv = newton(root, 0.5, fprime=vega, args=(S, K, T, r, q, t, M))
#     except Exception as e:
#         iv = 0        
#     ivs.append(iv)
    c += 1
    if T > (100 / 252) and abs(S / K - 1) <= 0.01:
        print(T)
        break

0.48412698412698413


In [25]:
S, K, T, r, q, t, M

(305.55, 303.0, 0.48412698412698413, 0.0018980546528298025, 0.0199, 1, 21.71)

In [26]:
brentq(root, 0, 100, args=(S, K, T, r, q, t, M))

-0.10127424496141657 21.71 0.0
302.6204270980067 21.71 100.0
298.93098812863883 21.71 7.205058027951015
43.842643906485705 21.71 0.5255335840724804
21.80372423643317 21.71 0.26084513191650577
21.710207655680335 21.71 0.2597290619548051
21.70999999706379 21.71 0.25972658371983454
21.710000000000008 21.71 0.2597265837548758
21.70999999991622 21.71 0.2597265837538757


/home/zquantz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in double_scalars
  after removing the cwd from sys.path.


0.2597265837548758